In [18]:
import os
import sys
import yaml

import pandas as pd

from network_wrangler import RoadwayNetwork
from network_wrangler import TransitNetwork
from network_wrangler import ProjectCard
from network_wrangler import Scenario
from network_wrangler import WranglerLogger

from lasso import ModelRoadwayNetwork

In [19]:
import warnings
warnings.filterwarnings("ignore")
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [28]:
import logging
logger = logging.getLogger("WranglerLogger")

# if you don't want to see so much detail, set to logging.INFO or DEBUG
logger.setLevel(logging.DEBUG)

## Read a Config File

Stored as `my_config` variable

In [29]:
MY_CONFIG_FILE = os.path.join(
    os.path.dirname(os.path.abspath('')), "examples", "settings","my_config.yaml"
)

with open(MY_CONFIG_FILE) as f:
        my_config = yaml.safe_load(f)
        
import json
print(json.dumps(my_config, indent=2))

## Alternatively this could be written in the notebook our selected via a notebook GUI

{
  "base_scenario": {
    "input_dir": "examples/stpaul",
    "shape_file_name": "shape.geojson",
    "link_file_name": "link.json",
    "node_file_name": "node.geojson",
    "transit_directory": "",
    "validate_network": false
  },
  "scenario": {
    "project_cards_filenames": [],
    "card_directory": "examples/stpaul/project_cards",
    "glob_search": "*attribute*.yml",
    "tags": []
  },
  "output": {
    "format": "geojson",
    "directory": "",
    "prefix": ""
  }
}


## Create a Base Scenario

In [30]:
base_project_path = os.path.dirname(os.path.abspath(''))

base_scenario = Scenario.create_base_scenario(
        my_config["base_scenario"]["shape_file_name"],
        my_config["base_scenario"]["link_file_name"],
        my_config["base_scenario"]["node_file_name"],
        base_dir=os.path.join(base_project_path,my_config["base_scenario"]["input_dir"]),
    )

#### Create project cards from projects that are explicitely specified in config


In [24]:
if len(my_config["scenario"]["project_cards_filenames"]) > 0:
    project_cards_list = [
        ProjectCard.read(filename, validate=False)
        for filename in my_config["scenario"]["project_cards_filenames"]
    ]
else: 
    project_cards_list = []
project_cards_list

[]

## Create Scenario

In [25]:
my_scenario = Scenario.create_scenario(
    base_scenario=base_scenario,
    card_directory=os.path.join(base_project_path,my_config["scenario"]["card_directory"]),
    tags=my_config["scenario"]["tags"],
    project_cards_list=project_cards_list,
    glob_search=my_config["scenario"]["glob_search"],
)

In [26]:
my_scenario.__dict__

{'road_net': <network_wrangler.RoadwayNetwork.RoadwayNetwork at 0x141c2e208>,
 'transit_net': <network_wrangler.TransitNetwork.TransitNetwork at 0x140219cf8>,
 'base_scenario': {'road_net': <network_wrangler.RoadwayNetwork.RoadwayNetwork at 0x141c2e208>,
  'transit_net': <network_wrangler.TransitNetwork.TransitNetwork at 0x140219cf8>},
 'applied_projects': [],
 'project_cards': [],
 'ordered_project_cards': OrderedDict(),
 'prerequisites': {},
 'corequisites': {},
 'conflicts': {},
 'requisites_checked': False,
 'conflicts_checked': False,
 'has_requisite_error': False,
 'has_conflict_error': False,
 'prerequisites_sorted': False}

### Apply all projects in scenario

In [ ]:
my_scenario.project_cards

In [ ]:
my_scenario.apply_all_projects()

In [13]:
my_scenario.applied_projects

[]

# Write out as MetCouncil Model Network

In [15]:
model_road_net = ModelRoadwayNetwork.from_RoadwayNetwork(
    my_scenario.road_net, parameters=my_config.get("my_parameters", {})
)

In [17]:
model_road_net.links_df.columns

Index(['model_link_id', 'osm_link_id', 'shstRefId', 'shstGeometryId', 'u', 'v',
       'A', 'B', 'access', 'area', 'bridge', 'highway', 'length', 'maxspeed',
       'name', 'oneway', 'tunnel', 'width', 'lanes', 'distance', 'HOV_access',
       'transit_priority', 'traveltime_assert', 'transit_access',
       'drive_access', 'walk_access', 'bike_access', 'transit_walk_access',
       'locationReferences', 'ref', 'geometry', 'area_type', 'county',
       'centroid_connector', 'mpo'],
      dtype='object')

In [16]:
model_road_net.write_roadway_as_shp()

KeyError: 'shstReferenceId'